<center>
<h1 style="background-color:orange; color:white" >Diabetes Classification EDA-ML</h1>

<center><img
src="https://res.cloudinary.com/grohealth/image/upload/f_auto,fl_lossy,q_auto/v1581692228/DCUK/Content/iStock-9217203841.jpg" style="width:90%;height:40%;">
</center>
    
<br>    
    
<div class="list-group" id="list-tab" role="tablist">
  <h3 class="list-group-item list-group-item-action active" style="background-color:orange; color:white" data-
    toggle="list"  role="tab" aria-controls="home">Content</h3>  
<ul>
        <li>EDA</li>
        <li>Modelling</li>
    </ul>


<h2>Variables</h2>
<ul>
<li>Pregnancies: Number of pregnancies </li>
<li>Glucose: Glucose </li>
<li>BloodPressure: Blood pressure </li>
<li>SkinThickness: Skin Thickness</li>
<li>Insulin: Insulin </li>
<li>BMI: Body mass index. </li>
<li>DiabetesPedigreeFunction: A function that calculates our probability of having diabetes based on our ancestry </li>
<li>Age: Age (years) </li>
<li>Outcome: Information whether the person has diabetes or not. Have the disease (1) or not (0) </li>

</ul>

In [ ]:
########################################################
#Importing Libraries and Plugins                       # 
########################################################
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score, confusion_matrix, classification_report, plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
import warnings
warnings.simplefilter(action="ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 170)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
#########################################
#Loading the Data Set                   #
#########################################
def dataset():
    df = pd.read_csv('../input/diabets/diabetes.csv')
    return df

df = dataset()

In [ ]:
#first 5 observations
df.head()

In [ ]:
#missing observations
df.isnull().sum()
    

In [ ]:
#row and column
df.shape

In [ ]:
####################################################
#                      EDA                         #
####################################################

In [ ]:
def dat_val(df,cat_th=10,car_th=20):
    cat_cols = [col for col in df.columns if df[col].dtypes =="O"]
    num_but_cat = [col for col in df.columns if df[col].nunique() < cat_th and
                  df[col].dtypes != "O"]
    cat_but_car = [col for col in df.columns if df[col].nunique() > car_th and
                  df[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in df.columns if df[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    
    
    print(f"Observation: {df.shape[0]}")
    print(f"Variables: {df.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    
    
    return cat_cols, cat_but_car, num_cols

In [ ]:
cat_cols, cat_but_car ,num_cols= dat_val(df)

In [ ]:
print(num_cols)

In [ ]:
for col in num_cols:
    plt.title('Histogram of numeric variables')
    plt.xlabel('Value')
    plt.ylabel(print(col))
    df[col].hist()
    plt.show() 

In [ ]:
###################################
#Analysis of Categorical Variables#
###################################


def categorical_analysis(df, col, plot=True):
    print(pd.DataFrame({col: df[col].value_counts(),
                       "Ratio": 100*df[col].value_counts()/len(df)}))
    
    if plot:
        sns.countplot(x=df[col], data=df)
        plt.show()

categorical_analysis(df, col, True)

In [ ]:
df.corr()

In [ ]:
# Correlation Matrix
f, ax = plt.subplots(figsize=[20, 15])
sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap="magma")
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

In [ ]:
# Distribution of Dependent Variable
sns.countplot('Outcome', data=df)
plt.show()

In [ ]:
list_num = []
for col in df.columns:
    if df[col].dtypes != 'O' and col not in ["Outcome"]:
        list_num.append(col)

#Outliers boxplot plot observed
def plot_outliers(df):
    for col in df.columns:
       if col in list_num:
                sns.boxplot(x=df[col])
                plt.title("BoxPlot Graphic Display")
                plt.show()
plot_outliers(df)


In [ ]:
#Target Analysis
def target_summary_with_num(dataframe, target, numerical_col):

     print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")

for col in num_cols:
    target_summary_with_num(df, "Outcome", col)

In [ ]:
#examined the breakdown chart to find the threshold value
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(df)
df_scores = clf.negative_outlier_factor_
scores = pd.DataFrame(np.sort(df_scores))
scores.plot(stacked=True, xlim=[0, 20], style='.-')
plt.show()


In [ ]:
esik_deger = np.sort(df_scores)[5] 
df[df_scores < esik_deger] #we have selected selection outliers based on threshold value
df[df_scores < esik_deger].shape 


In [ ]:
df[df_scores < esik_deger].index

In [ ]:
df.shape

In [ ]:
df = df.drop(axis=0, labels=df[df_scores < esik_deger].index)
df.head()

In [ ]:
df.shape

In [ ]:
for col in ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]:
    df[col].replace({0: np.nan},inplace=True)
df.dropna(inplace=True)
    

In [ ]:
df.shape

In [ ]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
clf.fit_predict(df)
df_scores = clf.negative_outlier_factor_
scores = pd.DataFrame(np.sort(df_scores))
scores.plot(stacked=True, xlim=[0, 20], style='.-')
plt.show()

esik_deger = np.sort(df_scores)[5] ##our threshold
df[df_scores < esik_deger] 
df[df_scores < esik_deger].shape 
df.describe().T

df[df_scores < esik_deger].index

df.drop(axis=0, labels=df[df_scores < esik_deger].index)
df = df.drop(axis=0, labels=df[df_scores < esik_deger].index)
df.head()

In [ ]:
 #person's relatives to have diabetes between 0-1
transformer = MinMaxScaler()
df["DiabetesPedigreeFunction"] = transformer.fit_transform(df[["DiabetesPedigreeFunction"]])
df.head()

In [ ]:
df["NEW_GLUCOSE"] = pd.cut(x=df["Glucose"], bins=[0, 140, 200, 300], labels=["Normal", "Prediabetes", "Diabetes"])

In [ ]:
df['NEW_BMI'] = pd.cut(x=df['BMI'], bins=[0, 18.5, 24.9, 29.9, 100],
                       labels=["Underweight", "Healthy", "Overweight", "Obese"])

In [ ]:
df["New_Insulin_Category"] = pd.cut(x=df["Insulin"],
                           bins=[0, 140, 200, df["Insulin"].max()],
                           labels=["Normal", "Hidden_diabetes", "Diabetes"])

df["New_Insulin_Category"] = df["New_Insulin_Category"].fillna("Normal")
df = df.drop("Insulin", axis=1)
df.head()

In [ ]:
df = pd.get_dummies(df, drop_first=True)
df.head()

In [ ]:
###########################
#        MODEL
############################
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=17)

cart_model = DecisionTreeClassifier(random_state=17).fit(X_train, y_train)
# train error
y_pred = cart_model.predict(X_train) 
y_prob = cart_model.predict_proba(X_train)[:, 1]
print(classification_report(y_train, y_pred))
roc_auc_score(y_train, y_prob) #1.0 overfit
print("#########################################################################")
# test error
y_pred = cart_model.predict(X_test)
y_prob = cart_model.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, y_prob) 

In [ ]:
cart_model = DecisionTreeClassifier(random_state=17)

cart_params = {'max_depth': range(1, 11),
               "min_samples_split": [2, 3, 4]}


cart_cv = GridSearchCV(cart_model, cart_params, cv=5, n_jobs=-1, verbose=True)
cart_cv.fit(X_train, y_train) 

In [ ]:
cart_tuned = DecisionTreeClassifier(**cart_cv.best_params_).fit(X_train, y_train)

# train hatası
y_pred = cart_tuned.predict(X_train)
y_prob = cart_tuned.predict_proba(X_train)[:, 1]
print(classification_report(y_train, y_pred))
roc_auc_score(y_train, y_prob)


#test  hatası
y_pred = cart_tuned.predict(X_test)
y_prob = cart_tuned.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, y_prob)

